In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score
import tensorflow as tf
warnings.filterwarnings('ignore')
import functions

from sklearn.model_selection import train_test_split
from functions import make_dataframe, ML, THRESHOLD

window_sizes = [10,20,30,60,80,120,240]
window_size = [60]
stride = 0.33333333

# for w in window_size:
#     STRIDE = int(w * stride)
#     KN = []
#     LGBM = []
#     DTC = []
#     RFC = []

#     for epoch in range(10):
#         df, _ = functions.make_dataframe(w, STRIDE)
#         X = df.iloc[:, :9].values
#         y = df['label'].values
#         X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2)
#         print(f"{epoch+1} Epochs...")
#         kn_ac, lgbm_ac, dtc_ac, clf_ac = ML(X_train, X_test, y_train, y_test)

#         KN.append(kn_ac)
#         LGBM.append(lgbm_ac)
#         DTC.append(dtc_ac)
#         RFC.append(clf_ac)

#     print("Window size : ", w, "Stride : ", STRIDE)
#     print("KN = ", KN)
#     print("LGBM = ", LGBM)
#     print("DTC = ", DTC)
#     print("RFC = ", RFC)

#     thres_X = df['MEAN_TEMP'].values
#     thres_Y = df['label'].values
#     print(accuracy_score(list(map(THRESHOLD, thres_X)), thres_Y))


In [2]:
from functions import make_dataframe, ML, THRESHOLD

df, df_temp_all = make_dataframe(60, 20)

In [3]:
df_temp_all

,TEMP,TIME
0,286.797,0
1,287.082,1
2,285.938,2
3,285.772,3
4,286.357,4
...,...,...
664791,865.029,664791
664792,864.985,664792
664793,865.048,664793
664794,865.040,664794


In [4]:
import torch.nn as nn
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_stacked_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_stacked_layers = num_stacked_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_stacked_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        batch_size = x.size(0)
        h0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        c0 = torch.zeros(self.num_stacked_layers, batch_size, self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

    
model = LSTM(1,4,1)
model.to(device)
lr = 0.001
num_epochs = 50
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(params = model.parameters(), lr = lr)
batch_size = 512

def train_one_epoch():
    model.train(True)
    print(f"Epoch : {epoch + 1}")
    training_loss = 0.0

    for batch_index, batch in enumerate(train_loader):
        X_batch, y_batch = batch[0].to(device), batch[1].to(device)

        output = model(X_batch)
        loss = criterion(output, y_batch)
        training_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_index % 1000 == 999:
            #avg_loss_cross_batches = training_loss / 1000
            #print("Batch {0}, Training Loss : {1}".format(batch_size+1, loss))
            pass
        training_loss = 0.0

def validate_one_epoch():
    model.train(False)
    validation_loss = 0.0

    for batch_index, batch in enumerate(test_loader):
        X_batch, y_batch = batch[0].to(device), batch[1].to(device)

        with torch.no_grad():
            output = model(X_batch)
            loss = criterion(output, y_batch)
            validation_loss += loss.item()

    avg_loss_across_batches = validation_loss / len(test_loader)

    #print('Val loss : {0}'.format(avg_loss_across_batches))

In [ ]:
from deeplearning_function import DEEPLEARNING, plot_prediction
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import r2_score

lookback_list = [5,10,20,30]
FB_list = ['front','back']
for lookback in lookback_list:
    for fb in FB_list:
        train_loader, test_loader, X_test, y_test = DEEPLEARNING(df_temp_all, lookback, batch_size = batch_size, front_or_back=fb)
        print("lookback size : ", lookback)
        print(fb)
        for epoch in range(num_epochs):
            train_one_epoch()
            validate_one_epoch()
            
        print("***********************************************")
        test_predictions, new_y_test = plot_prediction(model, X_test, y_test, lookback)

        plt.plot(new_y_test, label='Actual')
        plt.plot(test_predictions, label='Predicted')
        plt.xlabel('Time')
        plt.ylabel('Window size : ',lookback, fb ,'TEMP')
        plt.legend()
        plt.show()
        print(r2_score(new_y_test, test_predictions))
        print(mean_absolute_error(new_y_test, test_predictions))
        print(mean_squared_error(new_y_test, test_predictions))

lookback size :  5
front
Epoch : 1
Epoch : 2
Epoch : 3
Epoch : 4
Epoch : 5
Epoch : 6
Epoch : 7
Epoch : 8
Epoch : 9
Epoch : 10
Epoch : 11
Epoch : 12
Epoch : 13
Epoch : 14
Epoch : 15
Epoch : 16
Epoch : 17
Epoch : 18
Epoch : 19
Epoch : 20
Epoch : 21
Epoch : 22
Epoch : 23
Epoch : 24
Epoch : 25
Epoch : 26
Epoch : 27
Epoch : 28
Epoch : 29
Epoch : 30
Epoch : 31
Epoch : 32
Epoch : 33
Epoch : 34
Epoch : 35
Epoch : 36
Epoch : 37
Epoch : 38
Epoch : 39
Epoch : 40
Epoch : 41
Epoch : 42
Epoch : 43
Epoch : 44
Epoch : 45
Epoch : 46
Epoch : 47
Epoch : 48
Epoch : 49
Epoch : 50
***********************************************
0.9901588736480602
0.8583371806265796
1.3388545213075538
lookback size :  5
back
Epoch : 1
Epoch : 2
Epoch : 3
Epoch : 4


: 

# 시나리오

### #1 구간 1 -> 구간 6 -> 구간 7